In [1]:
import boto3
from datetime import datetime

In [ ]:
# Get the current local date and time. Define the variable 'today'
today = datetime.today()

In [ ]:
# Define a variable 'todays_date' and use it to format the date to a desired format
todays_date = today.strftime("%Y%m%d")

In [ ]:
# Connect with your S3 bucket by specifying your client[= variable (Search Google: boto3 aws s3)]
s3_client = boto3.client('s3')

In [ ]:
# Create a variable 'bucket_name' and set it to the name of your s3 bucket
bucket_name = "ovia-lambda-fn" 

In [ ]:
# Write a syntax to list objects in your s3 bucket ( Go to Google: boto3 aws list s3 objects)
list_objects_response = s3_client.list_objects_v2(Bucket=bucket_name)

In [ ]:
# Get only the contents from the dictionary using the 'get' method. Create a variable to store that: get_contents
get_contents = list_objects_response.get("Contents")

In [ ]:
# Get all the objects in your s3 bucket and store them in a variable
get_all_s3_objects_and_folder_names = []

# There are several objects so we use our 'For' loop to iterate
for item in get_contents:
    s3_object_name = item.get("Key")
    
# Now append it in the empty list
    get_all_s3_objects_and_folder_names.append(s3_object_name)

In [ ]:
# So we have extracted all the objects in our s3 bucket and stored them in this list
# Next we need to create a folder for today's date in our s3 bucket. if one does not exist. Remember the folder has to be named according to today's date

directory_name = todays_date + "/"
# That is, the variable 'directory_name' equals our previous variable 'todays_date' plus a forward slash. To create a folder in s3 bucket, you enter the folder name with a forward slash.

In [ ]:
# Next we create a folder in our s3 bucket, But first we check if it exists. So we use an 'If' statement

if directory_name not in get_all_s3_objects_and_folder_names:
    s3_client.put_object(Bucket=bucket_name, Key=(directory_name))
    
    # We call the s3 client to create a folder in our s3 bucket. So we look for the boto3 documentation to 'PUT' an object in our s3 bucket.
    

In [ ]:
# Next, we move any object that was uploaded today, into the folder having today's date:
# So we write our code for this, ensuring the two conditions are met:
# The last modified date must be today's date, and the object must not be a folder (i.e. with a forward slash), but a file.
# In S3, there's no method to move an object into a folder. So first, we copy the objects into the folder, then after that, we delete them.
# We start with a 'For' loop: We creates two variables: for 'last modified' and 'for key' which is our object name
for item in get_contents:
    object_creation_date = item.get("LastModified").strftime("%Y%m%d") + "/"
    object_name = item.get("Key")
    
    if object_creation_date == directory_name and "/" not in object_name:
        s3_client.copy_object(Bucket=bucket_name, CopySource=bucket_name+"/"+object_name, Key=directory_name+object_name)
        s3_client.delete_object(Bucket=bucket_name, Key=object_name)